In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('/content/database.sqlite')  # Replace '/path/to/your/database.db' with the path to your SQLite database file

# Getting a list of all tables in the database
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()

# Initializing a dictionary to hold DataFrames
dfs = {}

# Iterate over the list of tables
for table in tables:
    # Read each table into a DataFrame
    query = f"SELECT * FROM {table}"
    dfs[table] = pd.read_sql_query(query, conn)

# Close the connection to the database
conn.close()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])
# Separate features and target
X = df.drop('overall_rating', axis=1)
y = df['overall_rating']
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Apply preprocessing
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
### Step 2: Define and Compile the Neural Network Model
input_shape = X_train.shape[1]
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
### Step 3: Train the Model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
### Step 4: Evaluate the Model
mse = model.evaluate(X_test, y_test)
print(f'Test MSE: {mse}')

Epoch 1/100
199/199 [==============================] - 1s 4ms/step - loss: 1092.1594 - val_loss: 37.2778
Epoch 2/100
199/199 [==============================] - 1s 3ms/step - loss: 26.2454 - val_loss: 15.6768
Epoch 3/100
199/199 [==============================] - 1s 3ms/step - loss: 13.1043 - val_loss: 8.9330
Epoch 4/100
199/199 [==============================] - 1s 3ms/step - loss: 8.1540 - val_loss: 5.8772
Epoch 5/100
199/199 [==============================] - 1s 3ms/step - loss: 5.4014 - val_loss: 4.1964
Epoch 6/100
199/199 [==============================] - 1s 4ms/step - loss: 3.8555 - val_loss: 2.9281
Epoch 7/100
199/199 [==============================] - 1s 4ms/step - loss: 2.8531 - val_loss: 2.3859
Epoch 8/100
199/199 [==============================] - 1s 4ms/step - loss: 2.2256 - val_loss: 2.0363
Epoch 9/100
199/199 [==============================] - 1s 3ms/step - loss: 1.8507 - val_loss: 1.6900
Epoch 10/100
199/199 [==============================] - 1s 3ms/step - loss: 1.5777 -

In [ ]:
from kerastuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units_input', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(input_shape,)))
    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(Dense(hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                        activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning'
)

tuner.search(X_train, y_train, epochs=100, validation_split=0.2, verbose=1)


Trial 10 Complete [00h 02m 32s]
val_loss: 0.7306331992149353

Best val_loss So Far: 0.6439782381057739
Total elapsed time: 00h 19m 48s


In [ ]:
df.head()

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate,height,weight,age
0,60.142857,61.142857,50.142857,47.285714,46.285714,56.500000,38.000000,54.142857,50.000000,49.928571,...,12.857143,19.071429,10.357143,10.428571,right,medium,medium,175.26,154,43
1,69.380952,70.380952,36.428571,37.428571,75.333333,54.809524,20.428571,44.857143,30.428571,18.857143,...,6.095238,7.095238,9.095238,15.095238,right,medium,medium,187.96,174,41
2,69.285714,70.571429,42.238095,42.761905,66.666667,64.714286,28.761905,40.238095,51.761905,20.047619,...,15.904762,22.714286,15.142857,12.095238,left,medium,medium,180.34,154,41
3,71.133333,73.533333,61.866667,67.666667,67.800000,72.533333,67.133333,75.066667,79.800000,68.666667,...,15.666667,27.200000,16.333333,17.000000,right,medium,low,180.34,161,41
4,64.600000,70.533333,17.933333,16.066667,19.733333,21.200000,16.000000,17.933333,18.533333,18.733333,...,59.000000,59.866667,57.933333,69.533333,right,medium,medium,182.88,170,36


In [ ]:
def find_rows_with_higher_features_lower_rating(df, float_columns, rating_column):
    results = []
    # Iterate over all pairs of rows
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            # Count how many float64 features are higher in row i than in row j
            higher_count_i = sum(df.loc[i, col] > df.loc[j, col] for col in float_columns)
            higher_count_j = len(float_columns) - higher_count_i # Features higher in row j

            # Check majority and compare overall ratings
            if higher_count_i > higher_count_j and df.loc[i, rating_column] < df.loc[j, rating_column]:
                results.append((i, j))
            elif higher_count_j > higher_count_i and df.loc[j, rating_column] < df.loc[i, rating_column]:
                results.append((j, i))

    return results


float_cols = df.select_dtypes(include='float64').columns.drop('overall_rating')  # Assuming 'df_example' is your DataFrame
results = find_rows_with_higher_features_lower_rating(df, float_cols, 'overall_rating')

# Results would contain pairs of indices (i, j) where row i has a majority of higher feature values but a lower 'overall_rating' compared to row j